In [ ]:
import sys
sys.path.append('../python_packages_static/')
import pyemu
import os
import matplotlib.pyplot as plt

## run NOPTMAX=0 to get residuals, then load and reweight. This is a subjective step to balance the objective function for Monte Carlo and, ultimately, for history matching

# NB.... VERY OPEN TO REVISIONS TO THIS PARAGRAPH THAT WILL APPEAR IN EACH NOTEBOOK

## There are two paths for running the analysis of results in these notebooks. One is to run an ensemble locally and another is to run all the analysis without performing the PEST++ and model runs, using results from the journal article documenting this work. The selection is made using the `run_ensemble` variable below. If set to `True`, local runs are performed. If set to `False`, results from the journal article are used. This variable should be set appropriately to whichever folder these notebooks are being run in.

In [ ]:
run_ensemble=False

In [ ]:
if run_ensemble == True:
    input_dir = '../noptmax0_testing/' # we need to read from this directory to get the residuals from the noptmax 0 run
else:
    input_dir = '../output/noptmax0/' # we need to read from this directory to get the residuals from the noptmax 0 run
output_dir = '../run_data' # this is the folder we will actually use to set up PEST++ runs
pst_root = 'prior_mc'

### read in the PEST control file

In [ ]:
pst = pyemu.Pst(os.path.join(input_dir,'{}.pst'.format(pst_root)))

### make a quick pie chart showing the current distribution of the observation groups in the objective function

In [ ]:
pst.plot(kind='phi_pie')


### we can use `pyemu` functionality to assign new weights that adjust and honor whatever balance we seek

In [ ]:
new_proportions = pst.phi_components.copy()
new_proportions

In [ ]:
pst.observation_data

### here we assign proportions (that sum to 1.0) to the various groups. We want to retain the same total Phi so we multiply our proportions by the total Phi

In [ ]:
new_proportions['flux'] = 0.3*pst.phi
new_proportions['head'] = 0.2*pst.phi
new_proportions['land_surface'] = 0.5*pst.phi
new_proportions['budget'] = 0

In [ ]:
new_proportions

### using the `phi_components` dictionary, making a copy of it, and reassigning values, we can update the PST object using the `adjust_weights` function

In [ ]:
pst.adjust_weights(obsgrp_dict=new_proportions)

In [ ]:
pst.observation_data

### now we can see the updated pie chart

In [ ]:
pst.plot(kind='phi_pie')


In [ ]:
# set some values for ies
pst.pestpp_options["ies_num_reals"] = 500  
pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 3.0
pst.pestpp_options['ies_autoadaloc']=True
pst.pestpp_options['ies_num_threads']=4
pst.pestpp_options['ies_lambda_mults']=(0.1,1.0,10.0,100.0)
pst.pestpp_options['lambda_scale_fac'] = (0.75,0.9,1.0,1.1)
pst.pestpp_options['ies_subset_size']=10

# set SVD for some regularization
pst.svd_data.maxsing = 250

### using `pestpp-ies`, settting `noptmax=-1` and running in parallel will run a single pass through the initial ensemble, Monte Carlo style

In [ ]:
# set up for prior Monte Carlo
pst.control_data.noptmax = -1
pst.write(os.path.join(output_dir, '{}.pst'.format(pst_root)))